In [1]:
# imports
import os
os.sys.path.append(os.path.abspath("../.."))

import pandas as pd
import numpy as np

from project.model.ssd_model_300_xception import ssd_model_300_xception
from project.model.ssd_model_300_resnet import ssd_model_300_resnet
from project.model.ssd_model_300_vgg import ssd_model_300_vgg

from project.utils.data_bbox_match_hdf5 import load_ohc

from IPython.display import display

import h5py

import json

from tqdm import tqdm, trange

from keras.preprocessing import image

import project.download_content as content

from project.notebooks_utils import visuals
from matplotlib import pyplot as plt

from project.model.iou import iou

METAPATH = os.path.join(content.DATAPATH, 'METADATA')
MODELPATH = os.path.join(content.DATAPATH, 'MODEL')

CROSS_VAL_PREPROCESSED = os.path.join(content.DATAPATH,
                                      "MODEL",
                                      f"cross_val_data_preprocessed.h5")

TEST_PREPROCESSED = os.path.join(content.DATAPATH,
                                     "MODEL",
                                     f"test_data_preprocessed.h5")

Using TensorFlow backend.


In [2]:
df_cross_val = pd.read_hdf(CROSS_VAL_PREPROCESSED, key='X', mode='r')
df_cross_val

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside,Path,LabelSemantic,cx,cy,w,h,LabelID
0,0001eeaf4aed83f9,xclick,/m/0cmf2,1,0.022673,0.964201,0.071038,0.800546,0,0,0,0,0,project/data/VALIDATION/validation/0001eeaf4ae...,airplane,0.493437,0.435792,0.941527,0.729508,4
1,000595fe6fee6369,xclick,/m/02wbm,1,0.000000,1.000000,0.000000,1.000000,0,0,1,0,0,project/data/VALIDATION/validation/000595fe6fe...,food,0.500000,0.500000,1.000000,1.000000,199
2,000595fe6fee6369,xclick,/m/02xwb,1,0.141384,0.179676,0.676275,0.731707,0,0,0,0,0,project/data/VALIDATION/validation/000595fe6fe...,fruit,0.160530,0.703991,0.038292,0.055432,210
3,000595fe6fee6369,xclick,/m/02xwb,1,0.213549,0.253314,0.299335,0.354767,1,0,0,0,0,project/data/VALIDATION/validation/000595fe6fe...,fruit,0.233432,0.327051,0.039764,0.055432,210
4,000595fe6fee6369,xclick,/m/02xwb,1,0.232695,0.288660,0.490022,0.545455,1,0,0,0,0,project/data/VALIDATION/validation/000595fe6fe...,fruit,0.260677,0.517738,0.055965,0.055432,210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303975,ffff21932da3ed01,xclick,/m/03fp41,1,0.178125,0.710938,0.412500,0.579167,0,0,1,0,0,project/data/VALIDATION/validation/ffff21932da...,houseplant,0.444531,0.495833,0.532813,0.166667,258
303976,ffff21932da3ed01,xclick,/m/01g317,1,0.000000,0.015625,0.345833,0.414583,0,1,0,1,0,project/data/VALIDATION/validation/ffff21932da...,person,0.007812,0.380208,0.015625,0.068750,380
303977,ffff21932da3ed01,xclick,/m/01g317,1,0.062500,0.085938,0.431250,0.529167,1,0,0,0,0,project/data/VALIDATION/validation/ffff21932da...,person,0.074219,0.480208,0.023438,0.097917,380
303978,ffff21932da3ed01,xclick,/m/0c9ph5,1,0.323437,0.409375,0.464583,0.554167,0,0,1,0,0,project/data/VALIDATION/validation/ffff21932da...,flower,0.366406,0.509375,0.085938,0.089583,195


In [3]:
test_val = pd.read_hdf(TEST_PREPROCESSED, key='X', mode='r')
test_val

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside,Path,LabelSemantic,cx,cy,w,h,LabelID
0,000026e7ee790996,xclick,/m/07j7r,1,0.071875,0.145313,0.206250,0.391667,0,1,1,0,0,project/data/TEST/test/000026e7ee790996.jpg,tree,0.108594,0.298958,0.073438,0.185417,552
1,000026e7ee790996,xclick,/m/07j7r,1,0.439063,0.571875,0.264583,0.435417,0,1,1,0,0,project/data/TEST/test/000026e7ee790996.jpg,tree,0.505469,0.350000,0.132812,0.170833,552
2,000026e7ee790996,xclick,/m/07j7r,1,0.668750,1.000000,0.000000,0.552083,0,1,1,0,0,project/data/TEST/test/000026e7ee790996.jpg,tree,0.834375,0.276042,0.331250,0.552083,552
3,000062a39995e348,xclick,/m/015p6,1,0.206208,0.849224,0.154639,1.000000,0,0,0,0,0,project/data/TEST/test/000062a39995e348.jpg,bird,0.527716,0.577320,0.643016,0.845361,50
4,000062a39995e348,xclick,/m/05s2s,1,0.137472,0.376940,0.000000,0.883652,1,1,0,0,0,project/data/TEST/test/000062a39995e348.jpg,plant,0.257206,0.441826,0.239468,0.883652,392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937322,fffc6543b32da1dd,xclick,/m/04rky,1,0.903736,0.998563,0.347727,0.609091,1,1,0,0,0,project/data/TEST/test/fffc6543b32da1dd.jpg,mammal,0.951149,0.478409,0.094828,0.261364,320
937323,fffc6543b32da1dd,xclick,/m/0cnyhnx,1,0.012931,0.227011,0.368182,0.602273,0,0,0,0,0,project/data/TEST/test/fffc6543b32da1dd.jpg,bull,0.119971,0.485227,0.214080,0.234091,72
937324,fffc6543b32da1dd,xclick,/m/0cnyhnx,1,0.512931,0.632184,0.465909,0.600000,0,0,0,0,0,project/data/TEST/test/fffc6543b32da1dd.jpg,bull,0.572557,0.532955,0.119253,0.134091,72
937325,fffc6543b32da1dd,xclick,/m/04yx4,1,0.047414,0.995690,0.293182,0.784091,0,0,1,1,0,project/data/TEST/test/fffc6543b32da1dd.jpg,man,0.521552,0.538636,0.948276,0.490909,321


### FINAL EVALUATION

In [4]:
filepaths = ['AP-mobilenet.json',
             'AP-vgg.json',
             'AP-resnet.json',
             'AP-resnet-v2.json',
             'AP-xception.json',
             'AP-xception-v2.json']

weights = df_cross_val.groupby('LabelSemantic').count().iloc[:, 1].to_dict()

print(f"{'Architectures ':-<17}", 
      f"{'common ':-<8}",
      "weighted")
for path in filepaths:
    with open(os.path.join(MODELPATH, path)) as f:
        aps = json.load(f)
        weighted_sum = 0
        
        for k, v in aps.items():
            weighted_sum += v / weights[k] 
        
        print(f"{path.split('.')[0] + ' ':-<17}", 
              f"{(sum(float(ap) for ap in aps.values()) / len(weights)):.4f}{' ':-<3}",
              f"{weighted_sum:.4f}")

Architectures --- common - weighted
AP-mobilenet ---- 0.0012 -- 0.0002
AP-vgg ---------- 0.0036 -- 0.0020
AP-resnet ------- 0.0091 -- 0.0308
AP-resnet-v2 ---- 0.0067 -- 0.0353
AP-xception ----- 0.0174 -- 0.1199
AP-xception-v2 -- 0.0229 -- 0.1788
